### 5.3.1 - First Look at Spark / PySpark

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/02 16:05:24 WARN Utils: Your hostname, hp-computer resolves to a loopback address: 127.0.1.1; using 192.168.178.155 instead (on interface wlp0s20f3)
25/03/02 16:05:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/02 16:05:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget -c https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet -O ../data/fhvhv_tripdata_2021-01.parquet

--2025-03-02 16:05:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:20ae:de00:b:20a5:b140:21, 2600:9000:20ae:6e00:b:20a5:b140:21, 2600:9000:20ae:be00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:20ae:de00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Load the data to spark-dataframe:

In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet('../data/fhvhv_tripdata_2021-01.parquet')

In [5]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [6]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 1, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 1, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 1, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 1, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 1, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 1, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 18, 21), PULocationID=152, DO

Unlike `Pandas`, `Spark` does not do automatic type-inference and everything is represented as string. This can especially been seen when looking at the schema of the dataframe:

In [7]:
for col in df.schema:
    print(col)

StructField('hvfhs_license_num', StringType(), True)
StructField('dispatching_base_num', StringType(), True)
StructField('originating_base_num', StringType(), True)
StructField('request_datetime', TimestampType(), True)
StructField('on_scene_datetime', TimestampType(), True)
StructField('pickup_datetime', TimestampType(), True)
StructField('dropoff_datetime', TimestampType(), True)
StructField('PULocationID', LongType(), True)
StructField('DOLocationID', LongType(), True)
StructField('trip_miles', DoubleType(), True)
StructField('trip_time', LongType(), True)
StructField('base_passenger_fare', DoubleType(), True)
StructField('tolls', DoubleType(), True)
StructField('bcf', DoubleType(), True)
StructField('sales_tax', DoubleType(), True)
StructField('congestion_surcharge', DoubleType(), True)
StructField('airport_fee', DoubleType(), True)
StructField('tips', DoubleType(), True)
StructField('driver_pay', DoubleType(), True)
StructField('shared_request_flag', StringType(), True)
StructFiel

In [8]:
import pandas as pd

In [9]:
df_pandas = pd.read_parquet("../data/fhvhv_tripdata_2021-01.parquet", engine="pyarrow")
# df_pandas.dtypes

In [10]:
df_pandas.head(1001).to_parquet("head.parquet")
df_head = pd.read_parquet("head.parquet")

In [11]:
df_head.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[us]
on_scene_datetime       datetime64[us]
pickup_datetime         datetime64[us]
dropoff_datetime        datetime64[us]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [12]:
from pyspark.sql import types

In [13]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('originating_base_num', types.StringType(), True),  
    types.StructField('request_datetime', types.TimestampType(), True), 
    types.StructField('on_scene_datetime', types.TimestampType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.LongType(), True),  # Apparently problem with INT64
    types.StructField('DOLocationID', types.LongType(), True),  # Apparently problem with INT64
    types.StructField('trip_miles', types.DoubleType(), True), 
    types.StructField('trip_time', types.LongType(), True), 
    types.StructField('base_passenger_fare', types.DoubleType(), True), 
    types.StructField('tolls', types.DoubleType(), True), 
    types.StructField('bcf', types.DoubleType(), True), 
    types.StructField('sales_tax', types.DoubleType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True), 
    types.StructField('airport_fee', types.DoubleType(), True), 
    types.StructField('tips', types.DoubleType(), True), 
    types.StructField('driver_pay', types.DoubleType(), True), 
    types.StructField('shared_request_flag', types.StringType(), True), 
    types.StructField('shared_match_flag', types.StringType(), True), 
    types.StructField('access_a_ride_flag', types.StringType(), True), 
    types.StructField('wav_request_flag', types.StringType(), True), 
    types.StructField('wav_match_flag', types.StringType(), True)
])

In [14]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet("../data/fhvhv_tripdata_2021-01.parquet")

In [15]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 1, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 1, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 1, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 1, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 1, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 1, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 18, 21), PULocationID=152, DO

The columns of the spark dataframe are now of adequate type.

**Partitioning of Dataframes:**

- First: partition dataframe to utilize spark better
- Second: saving the partitions to parquet files

In [16]:
df = df.repartition(24)

In [18]:
df.write.parquet("fhvhv/2021/01/")

25/03/02 16:09:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


...

25/03/02 16:10:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/02 16:10:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/02 16:10:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


For replacing previous results use:
```python
df.write.parquet("fhvhv/2021/01/", mode="overwride")
```

### 5.3.2 - Spark DataFrames

In [19]:
df = spark.read.parquet("fhvhv/2021/01/")

In [20]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

**Access operations on spark dataframes**:

Some operations are executed lazy **`(Transformation)`**! This means that you can define an operation on a dataframe, but it is not executed until it is explicitely called with `.show()`. Other operations are executed immediately and return a result right away **`(Actions)`**. The latter category is mostly used when data is shown or taken from a dataframe.

In [21]:
# Lazy execution (select + filter dont execute the operations yet)
df.select("pickup_datetime", "dropoff_datetime", "PULocationID", "DOLocationID") \
  .filter(df.hvfhs_license_num == "HV0003") \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-11 19:40:22|2021-01-11 20:15:49|         262|         231|
|2021-01-05 16:13:22|2021-01-05 16:27:50|          61|         181|
|2021-01-31 19:42:09|2021-01-31 19:59:52|         232|           4|
|2021-01-27 23:24:36|2021-01-27 23:26:43|          68|          68|
|2021-01-30 09:35:46|2021-01-30 09:39:42|         256|         255|
|2021-01-16 03:25:35|2021-01-16 03:34:21|          89|          91|
|2021-01-11 12:58:23|2021-01-11 13:14:19|          97|          61|
|2021-01-03 08:44:58|2021-01-03 09:04:45|          26|         178|
|2021-01-14 19:52:00|2021-01-14 20:19:00|         181|         198|
|2021-01-08 21:35:35|2021-01-08 22:06:33|          76|          91|
|2021-01-15 14:49:48|2021-01-15 15:35:23|         246|          16|
|2021-01-27 11:37:56|2021-01-27 11:53:35|       

In [22]:
# eager execution (head is df-operation that requires concrete values immediately)
df.select("pickup_datetime", "dropoff_datetime", "PULocationID", "DOLocationID") \
  .filter(df.hvfhs_license_num == "HV0003") \
  .head(5)

[Row(pickup_datetime=datetime.datetime(2021, 1, 11, 19, 40, 22), dropoff_datetime=datetime.datetime(2021, 1, 11, 20, 15, 49), PULocationID=262, DOLocationID=231),
 Row(pickup_datetime=datetime.datetime(2021, 1, 5, 16, 13, 22), dropoff_datetime=datetime.datetime(2021, 1, 5, 16, 27, 50), PULocationID=61, DOLocationID=181),
 Row(pickup_datetime=datetime.datetime(2021, 1, 31, 19, 42, 9), dropoff_datetime=datetime.datetime(2021, 1, 31, 19, 59, 52), PULocationID=232, DOLocationID=4),
 Row(pickup_datetime=datetime.datetime(2021, 1, 27, 23, 24, 36), dropoff_datetime=datetime.datetime(2021, 1, 27, 23, 26, 43), PULocationID=68, DOLocationID=68),
 Row(pickup_datetime=datetime.datetime(2021, 1, 30, 9, 35, 46), dropoff_datetime=datetime.datetime(2021, 1, 30, 9, 39, 42), PULocationID=256, DOLocationID=255)]

SQL-equivalent for the operations:
```sql
SELECT 
    pickup_datetime, 
    dropoff_datetime, 
    PULocationID, 
    DOLocationID
FROM 
    df
WHERE 
    hvfhs_license_num = 'HV0003';
``` 

Using SQL-operations on Spark-Dataframes:

In [23]:
from pyspark.sql import functions as F

In [24]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

**Now**: Creating new columns for data-columns that are created from the `pickup_datetime` and `dropoff_datetime`.

In [25]:
df.withColumn("pickup_date", F.to_date(df.pickup_datetime)) \
  .withColumn("dropoff_date", F.to_date(df.dropoff_datetime)) \
  .select("pickup_date", "dropoff_date", "PULocationID", "DOLocationID") \
  .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-11|  2021-01-11|         262|         231|
| 2021-01-05|  2021-01-05|          61|         181|
| 2021-01-02|  2021-01-02|         100|           1|
| 2021-01-31|  2021-01-31|         232|           4|
| 2021-01-05|  2021-01-05|         162|           1|
| 2021-01-27|  2021-01-27|          68|          68|
| 2021-01-18|  2021-01-18|         205|         205|
| 2021-01-30|  2021-01-30|         256|         255|
| 2021-01-16|  2021-01-16|          89|          91|
| 2021-01-05|  2021-01-05|         132|         102|
| 2021-01-11|  2021-01-11|          97|          61|
| 2021-01-22|  2021-01-22|          79|          37|
| 2021-01-03|  2021-01-03|          26|         178|
| 2021-01-14|  2021-01-14|         181|         198|
| 2021-01-08|  2021-01-08|          76|          91|
| 2021-01-15|  2021-01-15|         246|       

It is also possible to write your own custom function that is applied to the dataframe. This is especially helpful if your requirement is hard to realize as a SQL-query: 

In [26]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f"s/{num:03x}"
    elif num % 3 == 0:
        return f"a/{num:03x}"
    else:
        return f"e/{num:03x}"

In [27]:
crazy_stuff("B02884") # Looks right

's/b44'

Using the user defined function as a function on Spark dataframe:

In [28]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [29]:
df.withColumn("pickup_date", F.to_date(df.pickup_datetime)) \
  .withColumn("dropoff_date", F.to_date(df.dropoff_datetime)) \
  .withColumn("base_id", crazy_stuff_udf(df.dispatching_base_num)) \
  .select("base_id", "pickup_date", "dropoff_date", "PULocationID", "DOLocationID") \
  .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/acc| 2021-01-11|  2021-01-11|         262|         231|
|  e/a39| 2021-01-05|  2021-01-05|          61|         181|
|  e/9ce| 2021-01-02|  2021-01-02|         100|           1|
|  e/b42| 2021-01-31|  2021-01-31|         232|           4|
|  s/af0| 2021-01-05|  2021-01-05|         162|           1|
|  a/b43| 2021-01-27|  2021-01-27|          68|          68|
|  e/9ce| 2021-01-18|  2021-01-18|         205|         205|
|  e/b35| 2021-01-30|  2021-01-30|         256|         255|
|  e/b3b| 2021-01-16|  2021-01-16|          89|          91|
|  e/9ce| 2021-01-05|  2021-01-05|         132|         102|
|  e/acc| 2021-01-11|  2021-01-11|          97|          61|
|  e/9ce| 2021-01-22|  2021-01-22|          79|          37|
|  e/b32| 2021-01-03|  2021-01-03|          26|         178|
|  a/b49| 2021-01-14|  2

### 5.3.4 SQL with Spark

In [30]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

**Loading green taxi data and yellow taxi data to spark dataframe**

In [31]:
# Loading data from all years and months of the green taxi dataset 
df_green = spark.read.parquet("../data/data/raw/green/*/*")

In [32]:
# Required when putting datasets together (green uses lpep, yellow uses tpep in datetime)
df_green = df_green \
    .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
    .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

In [33]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [34]:
df_green.show(5)

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|2019-12-18 16:52:30|2019-12-18 16:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.5|      0.01

In [35]:
# Load yellow taxi data from 2020 & 2021
df_yellow = spark.read.parquet("../data/data/raw/yellow/*/*")

In [36]:
# Required when putting datasets together (green uses lpep, yellow uses tpep in datetime)
df_yellow = df_yellow \
    .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
    .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

In [37]:
# The generic puckup and dropoff column-names are used now
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [38]:
df_yellow.show(5)

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1|2020-01-01 01:28:15|2020-01-01 01:33:03|            1.0|          1.2|       1.0|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.47|         0

**Creating one dataframe from both datasets**
- Only columns that are same in both datasets are retained

In [39]:
# Intersection of columns (original column order not preserved)
cols = set(df_green.columns) & set(df_yellow.columns)
cols

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [40]:
# Getting the intersecting columns in their original order
common_columns = []

yellow_cols = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_cols:
        common_columns.append(col)

In [41]:
from pyspark.sql import functions as F

In [42]:
# Adding `service_type` to the datasets s.t. the data 
# can be associated with the original dataset
df_green_sel = df_green \
    .select(common_columns) \
    .withColumn("service_type", F.lit("green"))

In [44]:
# Adding `service_type` to the datasets s.t. the data can be associated with a specific origin
df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn("service_type", F.lit("yellow"))

Combining both dataframes into one:

In [45]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [46]:
df_trips_data.groupBy("service_type").count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



Calling the data with `SQL` commands:

In [47]:
# Creating a view of the spark dataframe to send a query to
df_trips_data.createOrReplaceTempView("trips_data")

SQL-Version of this spark code:
```python
df_trips_data.groupBy("service_type").count().show()
```

In [48]:
spark.sql("""
SELECT 
    service_type,
    COUNT(1) as count
FROM 
    trips_data
GROUP BY
    service_type
""").show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [ ]:
query = """
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
"""

df_result = spark.sql(query)

In [50]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|          19|2020-

Saving the result of the query:

In [54]:
# Saves the df to as many parquet files as the dataframe has partitions
df_result.coalesce(1).write.parquet(
    "../data/data/report/revenue",
    mode="overwrite"
)

To specify the number of partitions a dataframe should be saved to, the method `coalesce(numPartitions: int) -> DataFrame` can be used. This however only works if the dataframe is also partitioned. Otherwise the data will be saved to one file.

In [55]:
# Repartitioning is usually a costly operation 
# but is ok since the dataframe is not that big
num_part = 24
df_result = df_result.repartition(num_part)

In [56]:
df_result.coalesce(numPartitions=1).write.parquet("../data/data/report/revenue", mode="overwrite")
df_result.coalesce(numPartitions=num_part).write.parquet("../data/data/report/revenue_partitioned", mode="overwrite")